In [31]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy.linalg import cholesky

First get weather data and metadata for a desired location (latitude and logitude)

In [32]:
# change to desired values (currently Miami)
latitude = 25.783388
longitude = -80.189029

API_KEY = credentials.API_KEY # my personal NREL api key
email ='tobin.ford@nrel.gov' # replace these values with your appropriate information and remove or comment out first line (import credentials)

# reads NSRDB data 
WEATHER, METADATA = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, email, names='tmy')

print(METADATA)

{'Source': 'NSRDB', 'Location ID': '1060699', 'City': '-', 'State': '-', 'Country': '-', 'Latitude': 25.77, 'Longitude': -80.18, 'Time Zone': -5, 'Elevation': 0, 'Local Time Zone': -5, 'Dew Point Units': 'c', 'DHI Units': 'w/m2', 'DNI Units': 'w/m2', 'GHI Units': 'w/m2', 'Temperature Units': 'c', 'Pressure Units': 'mbar', 'Wind Direction Units': 'Degrees', 'Wind Speed Units': 'm/s', 'Surface Albedo Units': 'N/A', 'Version': '3.2.0'}


c:\ProgramData\anaconda3\lib\site-packages\pvlib\iotools\psm3.py:359: pvlibDeprecationWarning: PSM3 variable names will be renamed to pvlib conventions by default starting in pvlib 0.11.0. Specify map_variables=True to enable that behavior now, or specify map_variables=False to hide this warning.
  warnings.warn(


User has 3 parameters so our matrix will be 3 x 3

In [53]:
# users will enter parameters around here
# currently 3x3 but function works on NxN matricies
# sometimes fails due to some Lin Alg reason
# depends on input matrix
text = [[25, 15, -5], [15, 18, 0], [-5, 0, 11]]
A = np.array(text)

print(A)

post_cholesky = cholesky(A)
print()
print(post_cholesky)


[[25 15 -5]
 [15 18  0]
 [-5  0 11]]

[[ 5.  3. -1.]
 [ 0.  3.  1.]
 [ 0.  0.  3.]]
